# Cross Entropy Method
A Monte Carlo approach to combinatorial and continuous multi-extremal optimization and importance sampling.

### Import necessary libraries

In [ ]:
import matplotlib; matplotlib.use("tkagg")
from matplotlib import animation
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np

# Cross Entropy Class
Contains methods for performing optimization using the Cross Entropy Method and for plotting the results of the procedure.

In [ ]:
class CrossEntropy:
    """
    Optimize a function using the Cross Entropy method. Adapted from:
    http://www.cleveralgorithms.com/nature-inspired/probabilistic/cross_entropy.html
    """

    CURVE_DENSITY = 40


    def __init__(self, dimension: int, bounds: np.ndarray, max_iterations: int, num_samples: int, num_update: int, learning_rate: float, **kwargs):
        """
        Initialize parameters for Cross Entropy method of optimization.

        :param dimension: number of dimensions to optimize.
        :param bounds: list of tuples of upper/lower bounds. List is the same size as dimension.
        :param max_iterations: maximum number of iterations in performing optimization.
        :param num_samples: number of samples in Monte Carlo simulation.
        :param num_update: number of samples to consider when updating distribution.
        :param learning_rate: speed of convergence.
        """

        self.dimension = dimension
        self.bounds = bounds.astype(np.float64)
        self.max_iterations = max_iterations
        self.num_samples = num_samples
        self.num_update = num_update
        self.learning_rate = learning_rate

        # initialize results to empty, will be set after optimization
        self.bests = []
        self.averages = []

        # kwargs for compatibility of **params in main function
        self.kwargs = kwargs


    def optimize(self, objective_function: callable, do_print: bool = False) -> (np.ndarray, list):
        """
        Optimizes given objective function.

        :param objective_function: function to optimize
        :param do_print: toggle output on and off
        :return: best solution, list of individuals by iteration
        """

        # Randomly initialize means within given bounds
        means = self.bounds[:, 0] + np.multiply(
            np.random.rand(self.dimension),
            self.bounds[:, 1] - self.bounds[:, 0]
        )

        # Initialize standard deviations as range of bounds
        standard_deviations = self.bounds[:, 1] - self.bounds[:, 0]
        standard_deviations = standard_deviations.astype(np.float64)

        # Initialize current best to None
        best_individual = None
        best_cost = None

        # Intialize stuff for plotting
        best_by_iteration = []
        average_by_iteration = []

        samples_by_iteration = []

        # Iterate until convergence or until max iterations is reached
        for iteration in range(self.max_iterations):

            # Generate samples from a Gaussian distribution between bounds
            samples = np.asarray(
                [[max(min(np.random.normal(means[i], standard_deviations[i]), self.bounds[i, 1]), self.bounds[i, 0]) for i in range(self.dimension)] for _ in range(self.num_samples)]
            )
            sample_costs = objective_function(samples)

            # Sort samples and sample costs by index
            sorted_indices = sample_costs.argsort()
            samples = samples[sorted_indices]
            sample_costs = sample_costs[sorted_indices]

            # Update best individual if we have discovered a new best
            if best_individual is None or best_cost > sample_costs[0]:
                best_individual, best_cost = samples[0], sample_costs[0]

            # Select individuals to influence distribution
            selected_individuals = samples[:self.num_update]

            # Update distribution parameters
            for i in range(self.dimension):
                means[i] = self.learning_rate * means[i] + ((1.0-self.learning_rate) * np.mean(selected_individuals[:, i]))
                standard_deviations[i] = self.learning_rate * standard_deviations[i] + ((1.0-self.learning_rate) * np.std(selected_individuals[:, i]))

            # Display status of algorithm
            if do_print:
                print("Iteration {iteration}\t\tBest cost: {fitness}\t\tBest individual: {individual}".format(
                    iteration=str.zfill(str(iteration), 3),
                    fitness=repr(best_cost),
                    individual=repr(best_individual)
                ))

            samples_by_iteration.append(samples)
            best_by_iteration.append(best_cost)
            average_by_iteration.append(np.mean(sample_costs))

        self.bests.append(best_by_iteration)
        self.averages.append(average_by_iteration)

        return best_individual, samples_by_iteration


    def plot_reps(self, title, random_search_avg: list = None, random_search_best: list = None) -> None:
        """
        Plot all reps with global best solutions.

        :param title: suptitle of plot
        :param random_search_avg: average fitness value for random search across generations
        :param random_search_best: best fitness value for random search by generation
        :return: void
        """

        is_random_search = not (random_search_avg is None or random_search_best is None)

        plt.subplots(1, 2, figsize=(16, 8))
        plt.suptitle(title)
        plt.subplot(1, 2, 1)
        plt.title("Average Cost by Iteration")
        plt.xlabel("Iteration")
        plt.ylabel("Average Cost")

        num_reps = len(self.averages)

        # plot each rep
        for rep in range(num_reps):
            plt.plot(range(len(self.averages[rep])), self.averages[rep], label="Rep %d Average" % (rep + 1))

        if is_random_search:
            plt.plot(range(len(random_search_avg)), random_search_avg, label="Random Search Average")
            # plt.plot(range(len(random_search_best)), random_search_best, label="Random Search Best")

        plt.legend(loc="upper left")

        plt.subplot(1, 2, 2)
        plt.title("Lowest Cost by Iteration")
        plt.xlabel("Iteration")
        plt.ylabel("Best Cost")

        # plot each rep
        for rep in range(num_reps):
            plt.plot(range(len(self.bests[rep])), self.bests[rep], label="Rep %d Best" % (rep + 1))

        if is_random_search:
            plt.plot(range(len(random_search_best)), random_search_best, label="Random Search Best")

        plt.legend(loc="upper left")
        plt.show()

# Random Search class
Simply class for performing a random search of the given space.

In [ ]:
class RandomSearch:


    def __init__(self, dimension: int, bounds: np.ndarray, max_iterations: int, num_samples: int, **kwargs):
        """
        Initialize parameters for Random Search benchmark.

        :param dimension: number of dimensions to optimize.
        :param bounds: list of tuples of upper/lower bounds. List is the same size as dimension.
        :param max_iterations: maximum number of iterations in performing optimization.
        :param num_samples: number of samples in Monte Carlo simulation.
        :param num_update: number of samples to consider when updating distribution.
        :param learning_rate: speed of convergence.
        """

        self.dimension = dimension
        self.bounds = bounds.astype(np.float64)
        self.max_iterations = max_iterations
        self.num_samples = num_samples

        # unused, just for compatibility with initializing CE and RS with same **params
        self.kwargs = kwargs


    def optimize(self, objective_function: callable, do_print: bool = False) -> (np.ndarray, list):
        """
        Optimizes given objective function.

        :param objective_function: function to optimize
        :return: best solution, list of individuals by iteration
        """

        # Initialize current best to None
        best_individual = None
        best_cost = None

        # Intialize stuff for plotting
        best_by_iteration = []
        average_by_iteration = []

        samples_by_iteration = []


        # Iterate until convergence or until max iterations is reached
        for iteration in range(self.max_iterations):

            # Generate samples from a Gaussian distribution between bounds
            samples = np.asarray(
                [[max(min(self.bounds[i, 0] + np.random.rand()*(self.bounds[i, 1] - self.bounds[i, 0]), self.bounds[i, 1]), self.bounds[i, 0]) for
                  i in range(self.dimension)] for _ in range(self.num_samples)]
            )
            sample_costs = objective_function(samples)

            # Sort samples and sample costs by index
            sorted_indices = sample_costs.argsort()
            samples = samples[sorted_indices]
            sample_costs = sample_costs[sorted_indices]

            # Update best individual if we have discovered a new best
            if best_individual is None or best_cost > sample_costs[0]:
                best_individual, best_cost = samples[0], sample_costs[0]

            # Display status of algorithm
            if do_print:
                print("Iteration {iteration}\t\tBest cost: {fitness}\t\tBest individual: {individual}".format(
                    iteration=str.zfill(str(iteration), 3),
                    fitness=repr(best_cost),
                    individual=repr(best_individual)
                ))

            samples_by_iteration.append(samples)
            best_by_iteration.append(best_cost)
            average_by_iteration.append(np.mean(sample_costs))


        return best_individual, samples_by_iteration, best_by_iteration, average_by_iteration

# Benchmark Functions
Below are several functions that are popular in benchmarking single-objective optimization algorithms. These functions came from the page https://en.wikipedia.org/wiki/Test_functions_for_optimization

## Rastrigin Function
![Rastrigin Function](img/Rastrigin_function.png)
Defined by
$$f(x) = An + \sum_{i=1}^{n} \left[ x_i^2 - A\cos{(2\pi x_i)}\right]$$
In this instance, we use $A = 10$ and $n=2$ for 2-dimensional vector inputs. The global minimum of the Rastrigin function is $f(\textbf{0})=0$. The recommended search domain is $-5.12\le x, y \le 5.12$.

In [ ]:
def Rastrigin(x: np.ndarray, A: float = 10, n: int = 2) -> float:
    """
    Benchmark function with many local optima.
    Global minimum is at x = 0.
    Bounds: (-5.12, 5.12)

    :param x: input vector
    :param A: arbitrary constant, typically A=10
    :param n: number of dimensions
    :return: value of Rastrigin function
    """
    total = np.ones((x.shape[0]))*A*n
    for i in range(n):
        total += (np.power(x[:, i], 2) - A*np.cos(2*np.pi*x[:, i]))
    return total.T

## Ackley's Function
![Ackley's Function](img/ackley.jpg)
Defined by
$$f(x, y) = -20\exp{\left[-0.2\sqrt{0.5(x^2+y^2)}\right]}-\exp{\left[0.5(\cos{(2\pi x)} + \cos{(2\pi y)}\right]} + e + 20$$
The global minimum of Ackley's function is $f(0, 0)=0$. The recommended search domain is $-5\le x, y \le 5$.

In [ ]:
def Ackley(x: np.ndarray) -> float:
    """
    Ackley function for benchmarking.
    Global minimum at (0, 0).
    Bounds: (-5, 5)

    :param x: input vector, must be 2D
    :return: value of Ackley function
    """
    return -20*np.exp(-0.2*np.sqrt(0.5*(np.power(x[:, 0], 2) + np.power(x[:, 1], 2)))) - \
        np.exp(0.5*(np.cos(2*np.pi*x[:, 0]) + np.cos(2*np.pi*x[:, 1]))) + np.e + 20

## Easom Function
![Easom Function](img/easom.jpg)
Defined by
$$f(x, y) = -\cos{(x)}\cos{(y)}\exp{\left(-\left((x-\pi)^2 + (y-\pi)^2\right)\right)}$$
The global minimum of the Easom's function is $f(\pi, \pi)=-1$. The recommended search domain is $-100\le x, y \le 100$. This function really tests the ability of the optimization algorithm to thoroughly sample the search space, as no solution is distinct aside from those inside the hole around $(\pi, \pi)$.

In [ ]:
def Easom(x: np.ndarray) -> float:
    """
    Easom function for benchmarking.
    Global minimum at f(pi, pi) = -1
    Bounds: (-100, 100)

    :param x: input vector, must be 2D
    :return: value of Easom function
    """
    return -np.cos(x[:, 0])*np.cos(x[:, 1])*np.exp(-(np.power(x[:, 0] - np.pi, 2) + np.power(x[:, 1] - np.pi, 2)))


## Cross-in-Tray Function
![Cross-in-Tray Function](img/crosstray.jpg)
Defined by
$$f(x, y) = -0.0001\left[\left\lvert \sin{x} \sin{y} \exp\left( \left\lvert 100 - \frac{\sqrt{x^2+y^2}}{\pi} \right\rvert \right) \right\rvert+ 1\right]^{0.1}$$
The global minimum of the cross-in-tray function is $f(\pm 1.34941, \pm 1.34941) = -2.06261$. The recommended search domain is $-10\le x, y \le 10$.

In [ ]:
def Cross_in_Tray(x: np.ndarray) -> float:
    """
    Cross-in-tray function for benchmarking.
    Global minimums at f(+- 1.34941, +-1.34941) = -2.06261
    Bounds: (-10, 10)

    :param x: input vector, must be 2D
    :return: value of Cross_in_Tray function
    """
    return -0.0001*np.power(np.abs(np.sin(x[:, 0])*np.sin(x[:, 1])*np.exp(np.abs(100-np.sqrt(np.power(x[:, 0], 2) + np.power(x[:, 1], 2))/np.pi))) + 1, 0.1)


## Levi Function
![Levi Function](img/levi.jpg)
Defined by
$$f(x, y) = \sin^2(3\pi x) + (x-1)^2(1+\sin^2(3\pi y) + (y-1)^2(1+\sin^2(2\pi y)$$
The global minimum of the Levi function is $f(1, 1) = 0$. The recommended search domain is $-10\le x, y \le 10$.

In [ ]:
def Levi(x: np.ndarray) -> float:
    """
    Levi function for benchmarking.
    Global minimum at f(0, 0) = 0
    Bounds: (-5, 5)
    
    :param x: input vector, must be 2D
    :return: value of Levi function
    """
    return np.power(np.sin(3*np.pi*x[:, 0]), 2) + np.multiply(np.power(x[:, 0] - 1, 2), 1+np.power(np.sin(3*np.pi*x[:, 1]), 2))+np.multiply(np.power(x[:, 1]-1, 2), 1+np.power(np.sin(2*np.pi*x[:, 1]), 2))

In [ ]:
def animate(objective_function: callable, samples_by_iteration, bounds: np.ndarray, curve_density: int):

    # Precalculate surface points for plotting
    xs = np.linspace(bounds[0][0], bounds[0][1], curve_density).reshape(
        (1, curve_density))
    ys = np.linspace(bounds[1][0], bounds[1][1], curve_density).reshape(
        (1, curve_density))
    X, Y = np.meshgrid(xs, ys)
    Z = objective_function(np.vstack((np.ravel(X), np.ravel(Y))).T).reshape(X.shape)

    fig = plt.figure()
    samples = fig.add_subplot(111, projection="3d")
    samples.plot_surface(X, Y, Z, color=(1, 1, 0, 0.5))
    individuals = samples.scatter(samples_by_iteration[0][:, 0], samples_by_iteration[0][:, 1], objective_function(samples_by_iteration[0]), c=(0, 0, 0), s=100)


    def init():
        return samples,

    def iterate(i):
        nonlocal individuals
        individuals.remove()
        individuals = samples.scatter(samples_by_iteration[i][:, 0], samples_by_iteration[i][:, 1], objective_function(samples_by_iteration[i]), c=(0, 0, 0), s=100)
        return samples,

    anim = animation.FuncAnimation(fig, iterate, init_func=init, frames=len(samples_by_iteration), interval=40, blit=False)
    plt.show()

In [ ]:
objective_functions = {
    "Sphere": {"f": lambda x: np.sum((np.power(x, 2)), axis=1), "bounds": np.asarray([[-5, 5], [-5, 5]])},
    "Rastrigin": {"f": Rastrigin, "bounds": np.asarray([[-5, 5], [-5, 5]])},
    "Ackley": {"f": Ackley, "bounds": np.asarray([[-5.12, 5.12], [-5.12, 5.12]])},
    "Easom": {"f": Easom, "bounds": np.asarray([[-100, 100], [-100, 100]])},
    "Cross in Tray": {"f": Cross_in_Tray, "bounds": np.asarray([[-10, 10], [-10, 10]])},
    "Levi": {"f": Levi, "bounds": np.asarray([[-10, 10], [-10, 10]])}
}

objective_function = "Levi"

params = {
    "dimension": 2,
    "bounds": objective_functions[objective_function]["bounds"],
    "max_iterations": 100,
    "num_samples": 50,
    "num_update": 5,
    "learning_rate": 0.7,
    "num_reps": 1
}

CE = CrossEntropy(**params)
RS = RandomSearch(**params)

RS_best, RS_samples_by_iteration, RS_best_by_iteration, RS_average_by_iteration = RS.optimize(objective_functions[objective_function]["f"])
CE_bests, CE_averages = [], []

CE_best, CE_samples_by_iteration = None, None
for rep in range(params["num_reps"]):
    CE_best, CE_samples_by_iteration = CE.optimize(objective_functions[objective_function]["f"], do_print=False)

animate(objective_functions[objective_function]["f"], CE_samples_by_iteration, params["bounds"], CrossEntropy.CURVE_DENSITY)
#CE.plot_reps("Cross Entropy on %s Function (α=%.2f)" % (objective_function, params["learning_rate"]), RS_average_by_iteration, RS_best_by_iteration)